In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint
from pathlib import Path
from random import randint

import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import glmsingle
from glmsingle.glmsingle import GLM_single
import bids
from bids import BIDSLayout
from scipy.ndimage import zoom, binary_dilation
import h5py
import nibabel as nib
from einops import rearrange

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from tc2see import load_data
from utils import rand_cmap

In [2]:
dataset_root = Path('D:\\Datasets')
tc2see_version = 3 # [1, 2]
dataset_path = dataset_root / f"TC2See_v{tc2see_version}"
derivatives_path = dataset_path / 'derivatives_TC2See_prdgm'

In [4]:
tr = 2 # 1.97
subject = 'sub-04'
run_ids = [2, 5]
bold, stimulus_ids, mask, affine = load_data(
    derivatives_path / f'tc2see-v{tc2see_version}-bold.hdf5', 
    subject,
    tr_offset=6 / tr,
    run_normalize='linear_trend',
    interpolation=False,
    run_ids=run_ids
)
bold.shape

(132, 189156)

In [5]:
#nc_file = 'noise_ceiling/sub-01/sub-01__time-linspace_0_6_13__interpolation-false__noise-ceiling.nii.gz'
nc_file = f'noise_ceiling/{subject}/{subject}__run_ids_2-5__noise-ceiling.nii.gz'
nc_series = nib.load(derivatives_path / nc_file).get_fdata()
component = 3 #8
nc_volume = nc_series[..., component]
nc_volume[:, :, :5] = 0.
nc = nc_volume[mask]
nc.shape

nc_threshold = 20.
X = bold[:, nc > nc_threshold]
X_nan = np.isnan(X)
X[X_nan] = 0.
X_nan.sum()

0

In [6]:
model_name = 'ViT-B=32'
embedding_name = 'embedding'

with h5py.File(derivatives_path / f'{model_name}-features.hdf5', 'r') as f:
    Y = f[embedding_name][:]
Y_stimulus = Y[stimulus_ids]

In [ ]:
unique_birds

In [7]:
stimulus_images = h5py.File(derivatives_path / 'stimulus-images.hdf5', "r")
image_names = list(stimulus_images.keys())
image_names = [image_names[i] for i in stimulus_ids]
image_names = np.array(image_names)

images = [stimulus_images[name]['data'][:] for name in image_names]
bird_class = np.array([stimulus_images[name].attrs['class_name'] for name in image_names])
#bird_superclass = (bird_class > 131).astype(int)

unique_birds, unique_indices = np.unique(bird_class, return_index=True)
class_names = image_names[unique_indices]
cmap = rand_cmap(unique_birds.shape[0], verbose=False)

In [8]:
from sklearn.manifold import TSNE

tsne = TSNE(
    n_components=2, 
    metric='cosine', 
    init="pca", 
    #learning_rate="auto", 
    random_state=0,
    verbose=10,
)
tsne.fit_transform(X)
None

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 132 samples in 0.001s...
[t-SNE] Computed neighbors for 132 samples in 0.001s...
[t-SNE] Computed conditional probabilities for sample 132 / 132
[t-SNE] Mean sigma: 0.309462
[t-SNE] Computed conditional probabilities in 0.004s
[t-SNE] Iteration 50: error = 58.6869888, gradient norm = 0.5689864 (50 iterations in 0.021s)
[t-SNE] Iteration 100: error = 58.3411140, gradient norm = 0.5353738 (50 iterations in 0.020s)
[t-SNE] Iteration 150: error = 58.5383339, gradient norm = 0.5436873 (50 iterations in 0.018s)
[t-SNE] Iteration 200: error = 60.4231796, gradient norm = 0.5080736 (50 iterations in 0.019s)
[t-SNE] Iteration 250: error = 58.3563232, gradient norm = 0.5774248 (50 iterations in 0.019s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 58.356323
[t-SNE] Iteration 300: error = 0.9168817, gradient norm = 0.0078661 (50 iterations in 0.019s)
[t-SNE] Iteration 350: error = 0.6917804, gradient norm = 0.0054783 (

In [9]:
out_path = derivatives_path / 'tsne' / subject
out_path.mkdir(exist_ok=True, parents=True)

y = tsne.embedding_
plt.figure(figsize=(16, 16))
for c, class_name in zip(unique_birds, class_names):
    #y_class = y[c == unique_birds]
    name = class_name#[4:-2].replace('_', ' ')
    plt.scatter(y_class[:, 0], y_class[:, 1], label=name, marker='o')# if 'Sparrow' in name else 'v')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
#plt.savefig(out_path / f'{model_name}__{embedding_name}_scatter.png', bbox_inches='tight')
#plt.savefig(out_path / f'bold_scatter.png', bbox_inches='tight')
plt.show()

NameError: name 'y_class' is not defined

<Figure size 1152x1152 with 0 Axes>

In [10]:
#stimulus_images = h5py.File(derivatives_path / 'stimulus-images.hdf5', "r")
#images = [image['data'][:] for image in stimulus_images.values()]

image_size = 128
num_images = 50
S = image_size * num_images
full_image = np.zeros(shape=(S, S, 3), dtype=np.ubyte)

In [11]:
extent = 20
coords = np.linspace(-extent, extent, num_images)
grid = np.stack(np.meshgrid(coords, coords))
grid = rearrange(grid, 'd h w -> (h w) d')
grid.shape

(2500, 2)

In [12]:
from sklearn.neighbors import NearestNeighbors
neighbors = NearestNeighbors(metric='chebyshev')
neighbors.fit(y)

distances, ids = neighbors.kneighbors(grid, n_neighbors=1,)
distances = rearrange(distances, '(h w) d -> h w d', h=num_images)
ids = rearrange(ids, '(h w) d -> h w d', h=num_images)
ids.shape

(50, 50, 1)

In [13]:
import PIL
from PIL import Image

distance_threshold = extent / num_images
for i in tqdm(range(num_images)):
    for j in range(num_images):
        if distances[i, j] > distance_threshold:
            continue
        stimulus_id = ids[i, j, 0]
        stim_image = images[stimulus_id]
        stim_image = Image.fromarray(stim_image)
        stim_image = stim_image.resize(size=(image_size, image_size), resample=PIL.Image.LANCZOS)
        stim_image = np.array(stim_image)
        full_image[i * image_size:(i + 1) * image_size, j * image_size:(j + 1) * image_size] = stim_image
full_image = full_image[:, ::-1]

  0%|          | 0/50 [00:00<?, ?it/s]

C:\Users\Cefir\AppData\Local\Temp\ipykernel_28056\2617690204.py:12: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  stim_image = stim_image.resize(size=(image_size, image_size), resample=PIL.Image.LANCZOS)


In [ ]:
full_image.shape

In [ ]:
Image.fromarray(full_image).save(out_path / f'{model_name}__{embedding_name}.png')

In [14]:
Image.fromarray(full_image).save(out_path / f'bold__runs_2-5.png')